In [11]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedKFold
import gensim
from gensim import models
import time
import pickle
import json
import urllib
import requests
import os
import gzip
import zipfile
from bisect import bisect_left
from xgboost import XGBClassifier

global_random_state = 42
k_fold_splits = 2

# Source: https://stackoverflow.com/questions/212358/binary-search-bisection-in-python/212971
def binary_search(a, x, lo=0, hi=None):  # can't use a to specify default for hi
    hi = hi if hi is not None else len(a)  # hi defaults to len(a)
    pos = bisect_left(a, x, lo, hi)  # find insertion position
    return (pos if pos != hi and a[pos] == x else -1)  # don't walk off the end


In [12]:
print("Loading from pickle")
with open('/media/data/pubchem/kekulesmiles_tuple.pickle',"rb") as f:
    cid_keys, smile_values = pickle.load(f)
print("Data loaded")

Loading from pickle
Data loaded


In [13]:
assay_dir = "/media/data/pubchem/Data"

def build_single_assay_dataset(assay_num,exclude_mols) :
    
    #print("Processing assay: {0}".format(assay_num))
    # Round assay into down to nearest thousand
    assay_num_rounded_lower = assay_num - (assay_num % 1000) + 1
    assay_num_rounded_upper = assay_num_rounded_lower + 999
    expected_folder_name = "{0:0>7}_{1:0>7}".format(assay_num_rounded_lower,assay_num_rounded_upper)
    expected_name = "{0}.zip".format(expected_folder_name)
    expected_path = os.path.join(assay_dir,expected_name)

    archive = zipfile.ZipFile(expected_path, 'r')

    fingerprints = list()
    activities = list()
    mols = list()
    num_parsed = 0
    
    with archive.open(expected_folder_name + '/' + str(assay_num) + ".csv.gz") as f:
        with gzip.open(f) as g:

            df = pd.read_csv(g,usecols=["PUBCHEM_CID","PUBCHEM_ACTIVITY_OUTCOME"])
            df = df.dropna(subset=["PUBCHEM_ACTIVITY_OUTCOME","PUBCHEM_CID"])
            df["PUBCHEM_CID"] = df["PUBCHEM_CID"].astype(int)
            df["IS_ACTIVE"] = df["PUBCHEM_ACTIVITY_OUTCOME"].apply(lambda x: True if "Active" in x else False)

            df_active = df[df["IS_ACTIVE"] == True]
            df_inactive = df[df["IS_ACTIVE"] == False]
            num_active = len(df_active)
            num_inactive = len(df_inactive)
            #print("Active are: {}, Inactive are: {}".format(num_active,num_inactive))

            active_added_count = 0
            inactive_added_count = 0
            num_parsed = 0

            for index, row in df.iterrows() :

                is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"

                if not is_active and inactive_added_count >= num_active:
                    continue

                cid = int(row["PUBCHEM_CID"])
                cid_pos = binary_search(cid_keys,cid)

                if cid_pos == -1:
                    continue

                smiles_string = smile_values[cid_pos]
                mol = Chem.MolFromSmiles(smiles_string)

                if mol is None or (exclude_mols is not None and mol in exclude_mols):
                    continue

                fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                 useBondTypes=False,useFeatures=False)
                # From RDKit documentation
                arr = np.zeros((1,))
                DataStructs.ConvertToNumpyArray(fingerprint, arr)
                fingerprint = arr

                fingerprints.append(fingerprint)                
                activities.append(is_active)
                mols.append(mol)

                num_parsed = num_parsed + 1

                if is_active:
                    active_added_count = active_added_count + 1
                else :
                    inactive_added_count = inactive_added_count + 1


    X = fingerprints
    y = activities
    if len(X) != len(y) :
        print("Lengths do not match")
    return X, y, mols
    

def build_multi_assay_dataset(gene_symbol,max_num_assays,holdout_assay_num=None) :

    # Use an API call to find all related bioassays
    assays_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/target/genesymbol/{0}/aids/TXT".format(gene_symbol)
    r = requests.get(assays_url)
    relevant_assays = [int(x) for x in r.text.split('\n') if len(x) > 0]
    print("Found a total of: {0} assays linked to Gene Symbol {1}".format(len(relevant_assays),gene_symbol))

    # Each assay n is in a file starting from 0001 to 1000, etc

    fingerprints = list()
    activities = list()
    mols = list()

    i = 0
    assays_used = list()

    for assay_num in relevant_assays:

        if holdout_assay_num and assay_num == holdout_assay_num:
            continue
        
        if i >= max_num_assays:
            break
            
        fp, ac, mol = build_single_assay_dataset(assay_num,None)
        
        fingerprints.extend(fp)
        activities.extend(ac)
        mols.extend(mol)
        
        assays_used.append(assay_num)

        i = i + 1
        
    print(len(fingerprints),len(activities))
    if len(fingerprints) != len(activities) :
        raise Exception("Mismatch in x and y for: {0}".format(gene_symbol))
    return np.array(fingerprints), np.array(activities), relevant_assays, assays_used, mols

def score_transfer_learning(X, y, X_holdout, y_holdout):
    classifier = DummyClassifier(random_state=global_random_state)
    classifier.fit(X,y)
    start = time.time()
    y_pred = classifier.predict(X_holdout)
    elapsed = time.time() - start
    auc = roc_auc_score(y_holdout, y_pred, average='macro', sample_weight=None)
    print("roc_auc score of {0} with DummyClassifier".format(auc))
    
    classifier = RandomForestClassifier(random_state=global_random_state)
    classifier.fit(X,y)
    start = time.time()
    y_pred = classifier.predict(X_holdout)
    elapsed = time.time() - start
    auc = roc_auc_score(y_holdout, y_pred, average='macro', sample_weight=None)

    print("roc_auc score of {0} with RandomForestClassifier".format(auc))
    
    classifier = XGBClassifier(random_state=global_random_state)
    classifier.fit(X,y)
    start = time.time()
    y_pred = classifier.predict(X_holdout)
    elapsed = time.time() - start
    auc = roc_auc_score(y_holdout, y_pred, average='macro', sample_weight=None)

    print("roc_auc score of {0} with XGBClassifier".format(auc))
    

X, y, relevant_assays, assays_used, mols = build_multi_assay_dataset("PPARG",5,1032)

#print("Core dataset size is: {0}".format(len(X)))
X_holdout, y_holdout, mols = build_single_assay_dataset(1032,mols)
#print("Comparison dataset size is: {0}".format(len(X_holdout)))

X_holdout = np.array(X_holdout)
y_holdout = np.array(y_holdout)

score_transfer_learning(X,y,X_holdout,y_holdout)

X = None
y = None
assays_used = None
mols = None


Found a total of: 912 assays linked to Gene Symbol PPARG
4335 4335
roc_auc score of 0.5006612974730309 with DummyClassifier
roc_auc score of 0.9344059405940595 with RandomForestClassifier
roc_auc score of 0.8596017437564653 with XGBClassifier


In [14]:
def score_transfer_learning_for_gene(gene_symbol, ref_assay_size) :
    X, y, relevant_assays, assays_used, mols_multi = build_multi_assay_dataset(gene_symbol,ref_assay_size)
    print(len(X),len(y))
    # Find a workable holdoutassay
    X_h_all = list()
    y_h_all = list()
    i = 0
    while(X_holdout is None or len(X_h_all) < 10 and i < len(relevant_assays) - ref_assay_size - 1) :
        holdout_assay = relevant_assays[ref_assay_size + i]
        X_h, y_h, mols = build_single_assay_dataset(holdout_assay, mols_multi)
        #print("X_h len: {0}".format(len(X_h)))
        #print("y_h len: {0}".format(len(y_h)))
        #print("Single assay with: {0} has X_h: {1}, y_h: {2}".format(holdout_assay,len(X_h),len(y_h)))
        X_h_all.extend(X_h)
        y_h_all.extend(y_h)
        #print("X_h_all is: {0}, y_h_all: {1}".format(len(X_h_all),len(y_h_all)))
        mols_multi.extend(mols)
        i = i + 1
        
    
    #print("X: {0}, y: {1}, X_holdout: {2}, y_holdout: {3}".format(len(X),len(y),len(X_h_all),len(y_h_all)))
    print("Scoring transferability for gene: {0}".format(gene_symbol))

    if len(X_h_all) != len(y_h_all) :
        raise Exception("Misaligned axis")
    
    if len(X_h_all) < 10:
        print("Skipping gene: {0}".format(gene_symbol))
        return
    
    score_transfer_learning(X,y,X_h_all,y_h_all)

# The 10 most studied genes from https://www.nature.com/articles/d41586-017-07291-9 
gene_list = ["TP53","TNF","EGFR","VEGFA","ESR1"]
for gene_symbol in gene_list:
    score_transfer_learning_for_gene(gene_symbol,5)

Found a total of: 176 assays linked to Gene Symbol TP53
6014 6014
6014 6014
Scoring transferability for gene: TP53
roc_auc score of 0.5016257896692679 with DummyClassifier
roc_auc score of 0.7283537718320328 with RandomForestClassifier
roc_auc score of 0.7458193979933111 with XGBClassifier
Found a total of: 47 assays linked to Gene Symbol TNF
1354 1354
1354 1354
Scoring transferability for gene: TNF
roc_auc score of 0.45238095238095233 with DummyClassifier
roc_auc score of 0.40476190476190477 with RandomForestClassifier
roc_auc score of 0.32142857142857145 with XGBClassifier
Found a total of: 977 assays linked to Gene Symbol EGFR
37 37
37 37
Scoring transferability for gene: EGFR
roc_auc score of 0.3666666666666667 with DummyClassifier
roc_auc score of 0.6333333333333334 with RandomForestClassifier
roc_auc score of 0.5166666666666667 with XGBClassifier
Found a total of: 22 assays linked to Gene Symbol VEGFA
1 1
1 1
Scoring transferability for gene: VEGFA
roc_auc score of 0.5 with Dummy